If you're opening this Notebook on colab, you will probably need to install 🤗 Transformers and 🤗 Datasets. Uncomment the following cell and run it.

In [ ]:
! pip install transformers datasets huggingface_hub -q

If you're opening this notebook locally, make sure your environment has an install from the last version of those libraries.

To be able to share your model with the community and generate results like the one shown in the picture below via the inference API, there are a few more steps to follow.

First you have to store your authentication token from the Hugging Face website (sign up [here](https://huggingface.co/join) if you haven't already!) then uncomment the following cell and input your token.

Then you need to install Git-LFS and setup Git if you haven't already. Uncomment the following instructions and adapt with your name and email:

Make sure your version of Transformers is at least 4.16.0 since the functionality was introduced in that version:

In [ ]:
import transformers

print(transformers.__version__)

4.38.2


You can find a script version of this notebook to fine-tune your model in a distributed fashion using multiple GPUs or TPUs [here](https://github.com/huggingface/transformers/tree/main/examples/tensorflow/question-answering).

We also quickly upload some telemetry - this tells us which examples and software versions are getting used so we know where to prioritize our maintenance efforts. We don't collect (or care about) any personally identifiable information, but if you'd prefer not to be counted, feel free to skip this step or delete this cell entirely.

In [ ]:
#from transformers.utils import send_example_telemetry

#send_example_telemetry("question_answering_notebook", framework="tensorflow")

# Fine-tuning a model on a question-answering task

In this notebook, we will see how to fine-tune one of the [🤗 Transformers](https://github.com/huggingface/transformers) model to a question answering task, which is the task of extracting the answer to a question from a given context. We will see how to easily load a dataset for these kinds of tasks and use Keras to fine-tune a model on it. Note that this model **does not generate new text!** Instead, it selects a span of the input passage as the answer.

![Widget inference representing the QA task](images/question_answering.png)

This notebook is built to run on any question answering task with the same format as SQUAD (version 1 or 2), with any model checkpoint from the [Model Hub](https://huggingface.co/models) as long as that model has a version with a token classification head and a fast tokenizer (check on [this table](https://huggingface.co/transformers/index.html#bigtable) if this is the case). It might, however, need some small adjustments if you decide to use a different dataset than the one used here. Depending on your model and the GPU you are using, you might need to adjust the batch size to avoid out-of-memory errors. Set those three parameters, then the rest of the notebook should run smoothly:

In [ ]:
# This flag is the difference between SQUAD v1 or 2 (if you're using another dataset, it indicates if impossible
# answers are allowed or not).
squad_v2 = True
#model_checkpoint = "bert-base-uncased"
model_checkpoint = "ALOQAS/bert-large-uncased-finetuned-squad-v2"
#batch_size = 16
batch_size = 64
# max_length = 384  # The maximum length of a feature (question and context)
# doc_stride = 128  # The allowed overlap between two part of the context when splitting is performed.

## Loading the dataset

We will use the [🤗 Datasets](https://github.com/huggingface/datasets) library to download the data and get the metric we need to use for evaluation (to compare our model to the benchmark). This can be easily done with the functions `load_dataset` and `load_metric`.  

In [ ]:
from datasets import load_dataset, load_metric

For our example here, we'll use the [SQUAD dataset](https://rajpurkar.github.io/SQuAD-explorer/). The notebook should work with any question answering dataset in the 🤗 Datasets library. If you're using your own dataset in a JSON or CSV file (see the [Datasets documentation](https://huggingface.co/docs/datasets/loading_datasets.html#from-local-files) on how to load them), it might need some adjustments to the column names.

**DATASETS FOR EVALUATION**

- **squad_v2**
  - train/**validation**
- **covid_qa_deepset** : https://huggingface.co/datasets/covid_qa_deepset
  - **train**
- **cuad** : https://huggingface.co/datasets/cuad
  - train/**test**
- **subjqa** : https://huggingface.co/datasets/subjqa
  - train/**validation**/test
  - multiple subsets : books/ **movies**/ restaurants/electronics/grocery/tripdavisor
- **TriviaQA_SQuAD** : https://huggingface.co/datasets/Kkordik/TriviaQA_SQuAD
  - **train**
- **coqa** : https://huggingface.co/datasets/stanfordnlp/coqa
  - train/validation
  - different format, needs to be adapted for format context / question / answers

In [ ]:
#dataset_split = "train"
#dataset_split = "test"
dataset_split = "validation"

In [ ]:
#datasets = load_dataset("squad_v2" if squad_v2 else "squad")
#datasets = load_dataset("covid_qa_deepset")
#datasets = load_dataset("Kkordik/TriviaQA_SQuAD")
#datasets = load_dataset("cuad")
# datasets = load_dataset("subjqa", "books")
# datasets = load_dataset("subjqa", "movies")
# datasets = load_dataset("subjqa", "restaurants")
# datasets = load_dataset("subjqa", "electronics")
# datasets = load_dataset("subjqa", "grocery")
datasets = load_dataset("subjqa", "tripadvisor")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/1165 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/512 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/230 [00:00<?, ? examples/s]

The `datasets` object itself is [`DatasetDict`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasetdict), which contains one key for the training, validation and test set.

In [ ]:
datasets

DatasetDict({
    train: Dataset({
        features: ['domain', 'nn_mod', 'nn_asp', 'query_mod', 'query_asp', 'q_reviews_id', 'question_subj_level', 'ques_subj_score', 'is_ques_subjective', 'review_id', 'id', 'title', 'context', 'question', 'answers'],
        num_rows: 1165
    })
    test: Dataset({
        features: ['domain', 'nn_mod', 'nn_asp', 'query_mod', 'query_asp', 'q_reviews_id', 'question_subj_level', 'ques_subj_score', 'is_ques_subjective', 'review_id', 'id', 'title', 'context', 'question', 'answers'],
        num_rows: 512
    })
    validation: Dataset({
        features: ['domain', 'nn_mod', 'nn_asp', 'query_mod', 'query_asp', 'q_reviews_id', 'question_subj_level', 'ques_subj_score', 'is_ques_subjective', 'review_id', 'id', 'title', 'context', 'question', 'answers'],
        num_rows: 230
    })
})

We can see the training, validation and test sets all have a column for the context, the question and the answers to those questions.

To access an actual element, you need to select a split first, then give an index:

In [ ]:
datasets["train"][0]

{'domain': 'tripadvisor',
 'nn_mod': 'awesome',
 'nn_asp': 'place',
 'query_mod': 'amazing',
 'query_asp': 'place',
 'q_reviews_id': '8ff9fa9cd9d246eeb73b26dbc9aa3b2e',
 'question_subj_level': 5,
 'ques_subj_score': 1.0,
 'is_ques_subjective': True,
 'review_id': 'tripadvisor_review_18201',
 'id': '074ee3857fe4a252088ae43606bd4cb9',
 'title': 'usa_san francisco_abigail_hotel',
 'context': "After reading the reviews I was expecting the worst... the hotel is comfortable, large rooms and friendly staff. Free breakfast was included which is a lot more than what we got in other much more expensive hotels we stayed in in California. The area which the hotel is in is indeed ghetto, drug dealers and users but they keep themselves to themselves. A bus or taxi is advisable to the centre as it is a distance to the main attractions. It is a budget hotel and doesn't claim to be anything else, worth the money and if you are travelling it is ideal. San Fran is really awesome ( least place we were loo

We can see the answers are indicated by their start position in the text (here at character 515) and their full text, which is a substring of the context as we mentioned above.

To get a sense of what the data looks like, the following function will show some examples picked randomly from the dataset and decoded back to strings.

In [ ]:
from datasets import ClassLabel, Sequence
import random
import pandas as pd
from IPython.display import display, HTML


def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(
        dataset
    ), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset) - 1)
        while pick in picks:
            pick = random.randint(0, len(dataset) - 1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(
                lambda x: [typ.feature.names[i] for i in x]
            )
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(datasets["train"])

,domain,nn_mod,nn_asp,query_mod,query_asp,q_reviews_id,question_subj_level,ques_subj_score,is_ques_subjective,review_id,id,title,context,question,answers
0,tripadvisor,complimentary,breakfast,great,continental breakfast,5062ada175042a0be5c96644049156ba,5,0.000000,False,tripadvisor_review_629,2a0b1ed978b8549856a6225f2bca0092,usa_san francisco_the_inn_at_union_square,"I really enjoyed staying at The Inn. I have stayed there twice &amp; each time was very exceptional service. The staff is friendly, helpful &amp; professional. The hotel gave me a very warming and cozy feeling. The rooms are clean &amp; updated. There's complimentary continental breakfast in the mornings, and snacks &amp; wine in the evenings. It's great for those who just want to sit down and relax next to the fireplace and read the paper after a full day out in the city! The Inn is very convenient to tons of shopping &amp; restaurants, also the cable car line, definitely a great location. Overall, The Inn is one of my overall favorites and I will return the next time I visit San Fran.",How is the continental breakfast?,"{'text': [], 'answer_start': [], 'answer_subj_level': [], 'ans_subj_score': [], 'is_ans_subjective': []}"
1,tripadvisor,perfect,bed,perfect,stay,52f48f54978262207ad3833397df4eaf,1,0.000000,False,tripadvisor_review_13975,9ff528bc1a06fe3e62cbf967f1f730a5,usa_san francisco_the_westin_st_francis,"The hotel is beautiful. Our room was perfect and the beds were SO comfortable. Didn't miss a wink of sleep. The bathroom was roomy. The lights were a little dim in the bathroom but that was our only complaint. The service was excellent. Be sure to ask for a room in the tower as these are newer and more roomy. The glassed elevator will transport you up to your sleeping paradise. We will stay here on every trip back to San Francisco. It is also conveniently located to all of the stores and restaurants, all within easy walking distance. Perfecto!",How was the stay?,"{'text': [], 'answer_start': [], 'answer_subj_level': [], 'ans_subj_score': [], 'is_ans_subjective': []}"
2,tripadvisor,pricey,food,overpriced,food,eada647864e956ee7bb3749fe19644f7,1,0.000000,False,tripadvisor_review_8601,3a7a1af079e26418f6955ac73ce45251,usa_san francisco_westin_san_francisco_market_street,"My wife and I,stayed at the Westin for three days Feb 14-17, 2008 and found the hotel to be above average. Check-in was quick, and they gave us a choice of King bed or two Queen beds. Room was clean, safe didn't work, was fixed within 10 minutes. There was a large screen plasma TV. Extra coffee on request was provided. Bed was comfortable, linens were of good quality, made up daily. Hotel staff was prompt and courteous. Hotel was close to Union Square and cable car line, close to SF Museum of Modern Art, close to the Ferry Bld. Just a cable car ride away from Chinatown and Fisherman's Wharf. In house limo service provided for return to airport for $8 more than a cab. Driver was great. A few cons; Street noise was noticable . We were on the 9th floor, quieted down by 11:00. We had breakfast there; the food was good but as in general for the Westin, pricey. In room bar fridge overpriced but convenient, has an automated system and we were charged for one drink we didn't have. Stores are close, just around the corner. Area tips: We ate across the street at the California Pizza Kitchen (fusion style menu). Food was good and priced okay. We also ate at Lefty O'Doules, (big comfort food menu), by Union Square. Unique sports decor, self-serve food, good sized portions, fresh bread. Try it--priced under 10$ for good-sized sandwich and side. If you stay for more than a couple of days and don't have a car, get a transit pass. Lets you use cable cars and buses. Bart is extra. Car parking is expensive; 40$ at the hotel, 20$ across the street and the streets are tough to navigate. We give the hotel our recommendation and would stay there again.",How is the food?,"{'text': ['food was good'], 'answer_start': [815

## Preprocessing the training data

Before we can feed those texts to our model, we need to preprocess them. This is done by a 🤗 Transformers `Tokenizer` which will (as the name indicates) tokenize the inputs (including converting the tokens to their corresponding IDs in the pretrained vocabulary) and put it in a format the model expects, as well as generate the other inputs that model requires.

To do all of this, we instantiate our tokenizer with the `AutoTokenizer.from_pretrained` method, which will ensure:

- we get a tokenizer that corresponds to the model architecture we want to use,
- we download the vocabulary used when pretraining this specific checkpoint.

That vocabulary will be cached, so it's not downloaded again the next time we run the cell.

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

The following assertion ensures that our tokenizer is a fast tokenizer (backed by Rust) from the 🤗 Tokenizers library. Those fast tokenizers are available for almost all models, and we will need some of the special features they have for our preprocessing.

In [ ]:
import transformers

assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

You can check which type of models have a fast tokenizer available and which don't in the [big table of models](https://huggingface.co/transformers/index.html#bigtable).

You can directly call this tokenizer on two sentences (one for the answer, one for the context):

In [ ]:
tokenizer("What is your name?", "My name is Sylvain.")

{'input_ids': [101, 2054, 2003, 2115, 2171, 1029, 102, 2026, 2171, 2003, 25353, 22144, 2378, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

Depending on the model you selected, you will see different keys in the dictionary returned by the cell above. They don't matter much for what we're doing here (just know they are required by the model we will instantiate later), you can learn more about them in [this tutorial](https://huggingface.co/transformers/preprocessing.html) if you're interested.

Now one specific thing for the preprocessing in question answering is how to deal with very long documents. We usually truncate them in other tasks, when they are longer than the model maximum sentence length, but here, removing part of the the context might result in losing the answer we are looking for. To deal with this, we will allow one (long) example in our dataset to give several input features, each of length shorter than the maximum length of the model (or the one we set as a hyper-parameter). Also, just in case the answer lies at the point we split a long context, we allow some overlap between the features we generate controlled by the hyper-parameter `doc_stride`:

In [ ]:
max_length = 384  # The maximum length of a feature (question and context)
doc_stride = 128  # The allowed overlap between two part of the context when splitting is performed.

Let's find one long example in our dataset:

In [ ]:
for i, example in enumerate(datasets["train"]):
    if len(tokenizer(example["question"], example["context"])["input_ids"]) > 384:
        break
example = datasets["train"][i]

Token indices sequence length is longer than the specified maximum sequence length for this model (607 > 512). Running this sequence through the model will result in indexing errors


Without any truncation, we get the following length for the input IDs:

In [ ]:
len(tokenizer(example["question"], example["context"])["input_ids"])

607

Now, if we just truncate, we will lose information (and possibly the answer to our question):

In [ ]:
len(
    tokenizer(
        example["question"],
        example["context"],
        max_length=max_length,
        truncation="only_second",
    )["input_ids"]
)

384

Note that we never want to truncate the question, only the context, and so we use the `only_second` truncation method. Our tokenizer can automatically return a list of features capped by a certain maximum length, with the overlap we talked about above, we just have to tell it to do so with `return_overflowing_tokens=True` and by passing the stride:

In [ ]:
tokenized_example = tokenizer(
    example["question"],
    example["context"],
    max_length=max_length,
    truncation="only_second",
    return_overflowing_tokens=True,
    stride=doc_stride,
)

Now we don't have one list of `input_ids`, but several:

In [ ]:
[len(x) for x in tokenized_example["input_ids"]]

[384, 360]

And if we decode them, we can see the overlap:

In [ ]:
for x in tokenized_example["input_ids"][:2]:
    print(tokenizer.decode(x))

[CLS] is the sink normal size? [SEP] my wife and i visited the andrews hotel for three nights as an anniversary getaway, leaving our young child at home. the hotel was a perfect launching off point for our adventures in san francisco. the & quot ; old world & quot ; style and charm of the hotel made it a great place for a romantic long weekend. the staff was professional, the room very comfortable despite its modest size, and the location excellent. having read other tripadvisor reviews, we booked a & quot ; comfy queen & quot ; and requested an upper - level room if possible. upon arrival, we were given room 709, back and away from the elevator on the hotel's top floor. it was nicely but simply decorated, with a tv monitor and dvd player in the room. the bed was on the soft side but quite comfortable. there was a small desk and dresser, and a nice leather chair in the corner. if we had wanted to, it would have been a nice quiet spot to spend a little time reading or something. the bat

It's going to take some work to properly label the answers here: we need to find in which of those features the answer actually is, and where exactly in that feature. The models we will use require the start and end positions of these answers in the tokens, so we will also need to to map parts of the original context to some tokens. Thankfully, the tokenizer we're using can help us with that by returning an `offset_mapping`:

In [ ]:
tokenized_example = tokenizer(
    example["question"],
    example["context"],
    max_length=max_length,
    truncation="only_second",
    return_overflowing_tokens=True,
    return_offsets_mapping=True,
    stride=doc_stride,
)
print(tokenized_example["offset_mapping"][0][:100])

[(0, 0), (0, 2), (3, 6), (7, 11), (12, 18), (19, 23), (23, 24), (0, 0), (0, 2), (3, 7), (8, 11), (12, 13), (14, 21), (22, 25), (26, 33), (34, 39), (40, 43), (44, 49), (50, 56), (57, 59), (60, 62), (63, 74), (75, 78), (78, 82), (82, 83), (84, 91), (92, 95), (96, 101), (102, 107), (108, 110), (111, 115), (115, 116), (117, 120), (121, 126), (127, 130), (131, 132), (133, 140), (141, 150), (151, 154), (155, 160), (161, 164), (165, 168), (169, 179), (180, 182), (183, 186), (187, 196), (196, 197), (198, 201), (202, 203), (203, 206), (206, 207), (207, 208), (208, 211), (212, 217), (217, 218), (218, 221), (221, 222), (222, 223), (224, 229), (230, 233), (234, 239), (240, 242), (243, 246), (247, 252), (253, 257), (258, 260), (261, 262), (263, 268), (269, 274), (275, 278), (279, 280), (281, 289), (290, 294), (295, 302), (302, 303), (304, 307), (308, 313), (314, 317), (318, 330), (330, 331), (332, 335), (336, 340), (341, 345), (346, 357), (358, 365), (366, 369), (370, 376), (377, 381), (381, 382), 

This gives the corresponding start and end character in the original text for each token in our input IDs. The very first token (`[CLS]`) has (0, 0) because it doesn't correspond to any part of the question/answer, then the second token is the same as the characters 0 to 3 of the question:

In [ ]:
first_token_id = tokenized_example["input_ids"][0][1]
offsets = tokenized_example["offset_mapping"][0][1]
print(
    tokenizer.convert_ids_to_tokens([first_token_id])[0],
    example["question"][offsets[0] : offsets[1]],
)

is Is


So we can use this mapping to find the position of the start and end tokens of our answer in a given feature. We just have to distinguish which parts of the offsets correspond to the question and which part correspond to the context, this is where the `sequence_ids` method of our `tokenized_example` can be useful:

In [ ]:
sequence_ids = tokenized_example.sequence_ids()
print(sequence_ids)

[None, 0, 0, 0, 0, 0, 0, None, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

It returns `None` for the special tokens, then 0 or 1 depending on whether the corresponding token comes from the first sentence past (the question) or the second (the context). Now with all of this, we can find the first and last token of the answer in one of our input feature (or if the answer is not in this feature):

In [ ]:
"""
answers = example["answers"]
start_char = answers["answer_start"][0]
end_char = start_char + len(answers["text"][0])

# Start token index of the current span in the text.
token_start_index = 0
while sequence_ids[token_start_index] != 1:
    token_start_index += 1

# End token index of the current span in the text.
token_end_index = len(tokenized_example["input_ids"][0]) - 1
while sequence_ids[token_end_index] != 1:
    token_end_index -= 1

# Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
offsets = tokenized_example["offset_mapping"][0]
if (
    offsets[token_start_index][0] <= start_char
    and offsets[token_end_index][1] >= end_char
):
    # Move the token_start_index and token_end_index to the two ends of the answer.
    # Note: we could go after the last offset if the answer is the last word (edge case).
    while (
        token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char
    ):
        token_start_index += 1
    start_position = token_start_index - 1
    while offsets[token_end_index][1] >= end_char:
        token_end_index -= 1
    end_position = token_end_index + 1
    print(start_position, end_position)
else:
    print("The answer is not in this feature.")
"""

'\nanswers = example["answers"]\nstart_char = answers["answer_start"][0]\nend_char = start_char + len(answers["text"][0])\n\n# Start token index of the current span in the text.\ntoken_start_index = 0\nwhile sequence_ids[token_start_index] != 1:\n    token_start_index += 1\n\n# End token index of the current span in the text.\ntoken_end_index = len(tokenized_example["input_ids"][0]) - 1\nwhile sequence_ids[token_end_index] != 1:\n    token_end_index -= 1\n\n# Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).\noffsets = tokenized_example["offset_mapping"][0]\nif (\n    offsets[token_start_index][0] <= start_char\n    and offsets[token_end_index][1] >= end_char\n):\n    # Move the token_start_index and token_end_index to the two ends of the answer.\n    # Note: we could go after the last offset if the answer is the last word (edge case).\n    while (\n        token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char

And we can double check that it is indeed the correct answer:

In [ ]:
"""
print(
    tokenizer.decode(
        tokenized_example["input_ids"][0][start_position : end_position + 1]
    )
)
print(answers["text"][0])
"""

'\nprint(\n    tokenizer.decode(\n        tokenized_example["input_ids"][0][start_position : end_position + 1]\n    )\n)\nprint(answers["text"][0])\n'

For this notebook to work with any kind of model, we need to account for the special case where the model expects padding on the left (in which case we switch the order of the question and the context):

In [ ]:
pad_on_right = tokenizer.padding_side == "right"

Now let's put everything together in one function we will apply to our training set. In the case of impossible answers (the answer is in another feature given by an example with a long context), we set the cls index for both the start and end position. We could also simply discard those examples from the training set if the flag `allow_impossible_answers` is `False`. Since the preprocessing is already complex enough as it is, we've kept is simple for this part.

In [ ]:
def prepare_train_features(examples):
    # Tokenize our examples with truncation and padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    # The offset mappings will give us a map from token to character position in the original context. This will
    # help us compute the start_positions and end_positions.
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # Let's label those examples!
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        # We will label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        # If no answers are given, set the cls_index as answer.
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
                token_start_index += 1

            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                token_end_index -= 1

            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (
                offsets[token_start_index][0] <= start_char
                and offsets[token_end_index][1] >= end_char
            ):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                # Note: we could go after the last offset if the answer is the last word (edge case).
                while (
                    token_start_index < len(offsets)
                    and offsets[token_start_index][0] <= start_char
                ):
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

This function works with one or several examples. In the case of several examples, the tokenizer will return a list of lists for each key:

In [ ]:
features = prepare_train_features(datasets["train"][:5])

To apply this function on all the sentences (or pairs of sentences) in our dataset, we just use the `map` method of the `dataset` object we created earlier. This will apply the function on all the elements of all the splits in `dataset`, so our training, validation and testing data will be preprocessed in one single command. Since our preprocessing changes the number of samples, we need to remove the old columns when applying it.

In [ ]:
tokenized_datasets = datasets.map(
    prepare_train_features, batched=True, remove_columns=datasets["train"].column_names
)

Map:   0%|          | 0/1165 [00:00<?, ? examples/s]

Map:   0%|          | 0/512 [00:00<?, ? examples/s]

Map:   0%|          | 0/230 [00:00<?, ? examples/s]

Even better, the results are automatically cached by the 🤗 Datasets library to avoid spending time on this step the next time you run your notebook. The 🤗 Datasets library is normally smart enough to detect when the function you pass to map has changed (and thus requires to not use the cache data). For instance, it will properly detect if you change the task in the first cell and rerun the notebook. 🤗 Datasets warns you when it uses cached files, you can pass `load_from_cache_file=False` in the call to `map` to not use the cached files and force the preprocessing to be applied again.

Note that we passed `batched=True` to encode the texts by batches together. This is to leverage the full benefit of the fast tokenizer we loaded earlier, which will use multi-threading to treat the texts in a batch concurrently.

## Loading the model

In [ ]:
from transformers import TFAutoModelForQuestionAnswering

model = TFAutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

All PyTorch model weights were used when initializing TFBertForQuestionAnswering.

All the weights of TFBertForQuestionAnswering were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForQuestionAnswering for predictions without further training.


## Evaluation

Evaluating our model will require a bit more work, as we will need to map the predictions of our model back to parts of the context. The model itself predicts logits for the start and end position of our answers: if we take a batch from our validation dataset, here is the output our model gives us:

In [ ]:
#=============AJOUT===================================
validation_set = model.prepare_tf_dataset(
    tokenized_datasets[dataset_split],
    shuffle=False,
    batch_size=batch_size,
)

In [ ]:
batch = next(iter(validation_set))
output = model.predict_on_batch(batch)
output.keys()

odict_keys(['start_logits', 'end_logits'])

The output of the model is a dict-like object that contains the loss (since we provided labels), the start and end logits. We won't need the loss for our predictions, let's have a look a the logits:

In [ ]:
output.start_logits.shape, output.end_logits.shape

((64, 384), (64, 384))

We have one logit for each feature and each token. The most obvious thing to predict an answer for each feature is to take the index for the maximum of the start logits as a start position and the index of the maximum of the end logits as an end position.

In [ ]:
import numpy as np

np.argmax(output.start_logits, -1), np.argmax(output.end_logits, -1)

(array([  0,   7,   0, 140,   0,   0,   0,   0, 108,   0,  90, 126, 120,
         20,   0,   0,  58,   0,  31,   0,  65,  10, 332, 203, 117, 315,
          0, 291,  58, 199, 208,  46,  27,   0,  93,  33,  30,  88,  82,
        112,   0, 252,  49,  44,  61, 248,  41,   0,  55,   0,   0,   0,
          0,   0,   0,   0, 109, 229,   0,   0,   0,   0,  35,   0]),
 array([  0,   0, 102, 142,   0,  30,  89,   0, 110, 153,  90, 130, 126,
         23,   0,   0,  61,   0,  31,   0,  66,  11, 332, 203,   0, 316,
          0, 296,  61, 212, 208,  47,  33,  44,  98,  48,  35,  92,  82,
        118,   0, 252,  49,  45,  62, 249,  43,   0,  56,   0,   0,   0,
          0,   0,   0,   0, 110, 233,   0,   0,   0,   0,  36,   0]))

This will work great in a lot of cases, but what if this prediction gives us something impossible: the start position could be greater than the end position, or point to a span of text in the question instead of the answer. In that case, we might want to look at the second best prediction to see if it gives a possible answer and select that instead.

However, picking the second best answer is not as easy as picking the best one: is it the second best index in the start logits with the best index in the end logits? Or the best index in the start logits with the second best index in the end logits? And if that second best answer is not possible either, it gets even trickier for the third best answer.


To classify our answers, we will use the score obtained by adding the start and end logits. We won't try to order all the possible answers and limit ourselves to with a hyper-parameter we call `n_best_size`. We'll pick the best indices in the start and end logits and gather all the answers this predicts. After checking if each one is valid, we will sort them by their score and keep the best one. Here is how we would do this on the first feature in the batch:

In [ ]:
n_best_size = 20

In [ ]:
import numpy as np

start_logits = output.start_logits[0]
end_logits = output.end_logits[0]
# Gather the indices the best start/end logits:
start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
valid_answers = []
for start_index in start_indexes:
    for end_index in end_indexes:
        if (
            start_index <= end_index
        ):  # We need to refine that test to check the answer is inside the context
            valid_answers.append(
                {
                    "score": start_logits[start_index] + end_logits[end_index],
                    "text": "",  # We need to find a way to get back the original substring corresponding to the answer in the context
                }
            )

And then we can sort the `valid_answers` according to their `score` and only keep the best one. The only point left is how to check a given span is inside the context (and not the question) and how to get back the text inside. To do this, we need to add two things to our validation features:
- the ID of the example that generated the feature (since each example can generate several features, as seen before);
- the offset mapping that will give us a map from token indices to character positions in the context.

That's why we will re-process the validation set with the following function, slightly different from `prepare_train_features`:

In [ ]:
def prepare_validation_features(examples):
    # Tokenize our examples with truncation and maybe padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

    # We keep the example_id that gave us this feature and we will store the offset mappings.
    tokenized_examples["example_id"] = []

    for i in range(len(tokenized_examples["input_ids"])):
        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1 if pad_on_right else 0

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["id"][sample_index])

        # Set to None the offset_mapping that are not part of the context so it's easy to determine if a token
        # position is part of the context or not.
        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]

    return tokenized_examples

And like before, we can apply that function to our validation set easily:

In [ ]:
validation_features = datasets[dataset_split].map(
    prepare_validation_features,
    batched=True,
    remove_columns=datasets[dataset_split].column_names,
)

Map:   0%|          | 0/230 [00:00<?, ? examples/s]

And turn the dataset into a `tf.data.Dataset` as before.

In [ ]:
validation_dataset = model.prepare_tf_dataset(
     validation_features,
     shuffle=False,
     batch_size=batch_size,
)

Now we can grab the predictions for all features by using the `model.predict` method:

In [ ]:
raw_predictions = model.predict(validation_dataset)

5/5 [==============================] - 22s 1s/step


In [ ]:
raw_predictions

TFQuestionAnsweringModelOutput(loss=None, start_logits=array([[ 4.7384024, -5.6387467, -7.540377 , ..., -9.265287 , -9.264123 ,
        -9.265836 ],
       [ 1.6410245, -8.129402 , -7.9985156, ..., -9.084703 , -9.099208 ,
        -9.111996 ],
       [ 2.3232257, -8.391501 , -8.094785 , ..., -9.23533  , -9.2339   ,
        -9.2367525],
       ...,
       [ 2.3092382, -8.045468 , -8.027033 , ..., -9.372764 , -9.376473 ,
        -9.392007 ],
       [ 2.687142 , -6.5240035, -6.8897023, ..., -9.414914 , -9.419925 ,
        -9.41846  ],
       [ 2.1585   , -5.88358  , -6.4522133, ..., -9.107867 , -9.128284 ,
        -9.141419 ]], dtype=float32), end_logits=array([[ 4.8797293, -6.5747986, -7.939644 , ..., -9.4104805, -9.409786 ,
        -9.406576 ],
       [ 1.9836681, -7.9048767, -8.061431 , ..., -9.5307255, -9.522852 ,
        -9.515922 ],
       [ 2.5219193, -7.9894056, -8.232361 , ..., -9.267832 , -9.262297 ,
        -9.259347 ],
       ...,
       [ 2.582941 , -8.247141 , -7.8501396, ...

We can now refine the test we had before: since we set `None` in the offset mappings when it corresponds to a part of the question, it's easy to check if an answer is fully inside the context. We also eliminate very long answers from our considerations (with an hyper-parameter we can tune)

In [ ]:
max_answer_length = 30

In [ ]:
start_logits = output.start_logits[0]
end_logits = output.end_logits[0]
offset_mapping = validation_features[0]["offset_mapping"]
# The first feature comes from the first example. For the more general case, we will need to be match the example_id to
# an example index
context = datasets[dataset_split][0]["context"]

# Gather the indices the best start/end logits:
start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
valid_answers = []
for start_index in start_indexes:
    for end_index in end_indexes:
        # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
        # to part of the input_ids that are not in the context.
        if (
            start_index >= len(offset_mapping)
            or end_index >= len(offset_mapping)
            or offset_mapping[start_index] is None
            or offset_mapping[end_index] is None
        ):
            continue
        # Don't consider answers with a length that is either < 0 or > max_answer_length.
        if end_index < start_index or end_index - start_index + 1 > max_answer_length:
            continue
        if (
            start_index <= end_index
        ):  # We need to refine that test to check the answer is inside the context
            start_char = offset_mapping[start_index][0]
            end_char = offset_mapping[end_index][1]
            valid_answers.append(
                {
                    "score": start_logits[start_index] + end_logits[end_index],
                    "text": context[start_char:end_char],
                }
            )

valid_answers = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[
    :n_best_size
]
valid_answers

[{'score': -4.29566,
  'text': 'The room I had was clean and well presented if a little old'},
 {'score': -4.3593435, 'text': '4 blocks'},
 {'score': -4.3634844,
  'text': '4 blocks). Slight drawback was the toaster setting off the fire alarm 2 out of 3 mornings'},
 {'score': -4.4212046,
  'text': 'pink paint effect that covers walls and all woodwork'},
 {'score': -4.47338, 'text': 'pink paint effect'},
 {'score': -4.6158986, 'text': 'pink paint effect that covers walls'},
 {'score': -4.7885065, 'text': 'clean and well presented if a little old'},
 {'score': -4.8463707, 'text': 'pink paint'},
 {'score': -4.9480095, 'text': '2 out of 3 mornings'},
 {'score': -4.9919157, 'text': 'pink'},
 {'score': -5.025022, 'text': '4 blocks). Slight drawback was the toaster'},
 {'score': -5.0335417,
  'text': 'The room I had was clean and well presented if a little old. If you can ignore the rather misguided pink paint effect'},
 {'score': -5.080699,
  'text': '4 blocks). Slight drawback was the toast

We can compare to the actual ground-truth answer:

In [ ]:
datasets[dataset_split][0]["answers"]

{'text': [],
 'answer_start': [],
 'answer_subj_level': [],
 'ans_subj_score': [],
 'is_ans_subjective': []}

Our model's most likely answer is correct!

As we mentioned in the code above, this was easy on the first feature because we knew it comes from the first example. For the other features, we will need a map between examples and their corresponding features. Also, since one example can give several features, we will need to gather together all the answers in all the features generated by a given example, then pick the best one. The following code builds a map from example index to its corresponding features indices:

In [ ]:
import collections

examples = datasets[dataset_split]
features = validation_features

example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
features_per_example = collections.defaultdict(list)
for i, feature in enumerate(features):
    features_per_example[example_id_to_index[feature["example_id"]]].append(i)

We're almost ready for our post-processing function. The last bit to deal with is the impossible answer (when `squad_v2 = True`). The code above only keeps answers that are inside the context, we need to also grab the score for the impossible answer (which has start and end indices corresponding to the index of the CLS token). When one example gives several features, we have to predict the impossible answer when all the features give a high score to the impossible answer (since one feature could predict the impossible answer just because the answer isn't in the part of the context it has access too), which is why the score of the impossible answer for one example is the *minimum* of the scores for the impossible answer in each feature generated by the example.

We then predict the impossible answer when that score is greater than the score of the best non-impossible answer. All combined together, this gives us this post-processing function:

In [ ]:
from tqdm.auto import tqdm

def postprocess_qa_predictions(
    examples,
    features,
    all_start_logits,
    all_end_logits,
    n_best_size=20,
    max_answer_length=30,
):
    # Build a map example to its corresponding features.
    example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)

    # The dictionaries we have to fill.
    predictions = collections.OrderedDict()

    # Logging.
    print(
        f"Post-processing {len(examples)} example predictions split into {len(features)} features."
    )

    # Let's loop over all the examples!
    for example_index, example in enumerate(tqdm(examples)):
        # Those are the indices of the features associated to the current example.
        feature_indices = features_per_example[example_index]

        min_null_score = None  # Only used if squad_v2 is True.
        valid_answers = []

        context = example["context"]
        # Looping through all the features associated to the current example.
        for feature_index in feature_indices:
            # We grab the predictions of the model for this feature.
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]
            # This is what will allow us to map some the positions in our logits to span of texts in the original
            # context.
            offset_mapping = features[feature_index]["offset_mapping"]

            # Update minimum null prediction.
            cls_index = features[feature_index]["input_ids"].index(
                tokenizer.cls_token_id
            )
            feature_null_score = start_logits[cls_index] + end_logits[cls_index]
            if min_null_score is None or min_null_score < feature_null_score:
                min_null_score = feature_null_score

            # Go through all possibilities for the `n_best_size` greater start and end logits.
            start_indexes = np.argsort(start_logits)[
                -1 : -n_best_size - 1 : -1
            ].tolist()
            end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
                    # to part of the input_ids that are not in the context.
                    if (
                        start_index >= len(offset_mapping)
                        or end_index >= len(offset_mapping)
                        or not offset_mapping[start_index]
                        or not offset_mapping[end_index]
                    ):
                        continue
                    # Don't consider answers with a length that is either < 0 or > max_answer_length.
                    if (
                        end_index < start_index
                        or end_index - start_index + 1 > max_answer_length
                    ):
                        continue
                    start_char = offset_mapping[start_index][0]
                    end_char = offset_mapping[end_index][1]
                    valid_answers.append(
                        {
                            "score": start_logits[start_index] + end_logits[end_index],
                            "text": context[start_char:end_char],
                        }
                    )

        if len(valid_answers) > 0:
            best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[
                0
            ]
        else:
            # In the very rare edge case we have not a single non-null prediction, we create a fake prediction to avoid
            # failure.
            best_answer = {"text": "", "score": 0.0}

        # Let's pick our final answer: the best one or the null answer (only for squad_v2)
        if not squad_v2:
            predictions[example["id"]] = best_answer["text"]
        else:
            answer = (
                best_answer["text"] if best_answer["score"] > min_null_score else ""
            )
            predictions[example["id"]] = answer

    return predictions

And we can apply our post-processing function to our raw predictions:

In [ ]:
final_predictions = postprocess_qa_predictions(
    datasets[dataset_split],
    validation_features,
    raw_predictions["start_logits"],
    raw_predictions["end_logits"],
)

Post-processing 230 example predictions split into 277 features.


  0%|          | 0/230 [00:00<?, ?it/s]

Then we can load the metric from the datasets library.

In [ ]:
metric = load_metric("squad_v2" if squad_v2 else "squad")

<ipython-input-47-69495f532713>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("squad_v2" if squad_v2 else "squad")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for squad_v2 contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.0/metrics/squad_v2/squad_v2.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


Then we can call compute on it. We just need to format predictions and labels a bit as it expects a list of dictionaries and not one big dictionary. In the case of squad_v2, we also have to set a `no_answer_probability` argument (which we set to 0.0 here as we have already set the answer to empty if we picked it).

In [ ]:
if squad_v2:
    formatted_predictions = [
        {"id": k, "prediction_text": v, "no_answer_probability": 0.0}
        for k, v in final_predictions.items()
    ]
else:
    formatted_predictions = [
        {"id": k, "prediction_text": v} for k, v in final_predictions.items()
    ]
references = [
    {"id": ex["id"], "answers": ex["answers"]} for ex in datasets[dataset_split]
]
metric.compute(predictions=formatted_predictions, references=references)

{'exact': 20.0,
 'f1': 34.13426260363784,
 'total': 230,
 'HasAns_exact': 5.633802816901408,
 'HasAns_f1': 28.527326752371167,
 'HasAns_total': 142,
 'NoAns_exact': 43.18181818181818,
 'NoAns_f1': 43.18181818181818,
 'NoAns_total': 88,
 'best_exact': 38.26086956521739,
 'best_exact_thresh': 0.0,
 'best_f1': 38.26086956521739,
 'best_f1_thresh': 0.0}

### Results on dataset "squad_v2"

- 'exact': 72.39956203150004
- 'f1': 75.82176186193121
- 'total': 11873
---
- 'HasAns_exact': 74.88191632928475
- 'HasAns_f1': 81.73612999101049
- 'HasAns_total': 5928
---
- 'NoAns_exact': 69.92430613961312
- 'NoAns_f1': 69.92430613961312
- 'NoAns_total': 5945
---
- 'best_exact': 72.39956203150004
- 'best_exact_thresh': 0.0
- 'best_f1': 75.82176186193124
- 'best_f1_thresh': 0.0

### Results on dataset "covid_qa_deepset"

- 'exact': 17.830609212481427
- 'f1': 29.522734309083905
- 'total': 2019
---
- 'HasAns_exact': 17.830609212481427
- 'HasAns_f1': 29.522734309083905
- 'HasAns_total': 2019
---
- 'best_exact': 17.830609212481427
- 'best_exact_thresh': 0.0
- 'best_f1': 29.522734309083905
- 'best_f1_thresh': 0.0

### Results on dataset "TriviaQA_SQuAD"

- 'exact': 24.738210696464964
- 'f1': 28.979069573681603
- 'total': 14229
---
- 'HasAns_exact': 19.170377314272322
- 'HasAns_f1': 23.884320050301984
- 'HasAns_total': 12801
---
- 'NoAns_exact': 74.64985994397759
- 'NoAns_f1': 74.64985994397759
- 'NoAns_total': 1428
---
- 'best_exact': 24.74523859723101
- 'best_exact_thresh': 0.0
- 'best_f1': 28.980575552417115
- 'best_f1_thresh': 0.0

### Results on dataset "cuad"

- 'exact': 68.84265901482544
- 'f1': 69.23192068217402
- 'total': 4182
---
- 'HasAns_exact': 0.9646302250803859
- 'HasAns_f1': 2.273225315797423
- 'HasAns_total': 1244
---
- 'NoAns_exact': 97.58339006126617
- 'NoAns_f1': 97.58339006126617
- 'NoAns_total': 2938
---
- 'best_exact': 70.3012912482066
- 'best_exact_thresh': 0.0
- 'best_f1': 70.30746717730344
- 'best_f1_thresh': 0.0

### Results on dataset "subjqa", subset "books"

#### Subset "books" (subjqa)

- 'exact': 39.84375
- 'f1': 44.99711124251565
- 'total': 256
---
- 'HasAns_exact': 2.2222222222222223
- 'HasAns_f1': 16.88067197871119
- 'HasAns_total': 90
---
- 'NoAns_exact': 60.24096385542169
- 'NoAns_f1': 60.24096385542169
- 'NoAns_total': 166
---
- 'best_exact': 64.84375
- 'best_exact_thresh': 0.0
- 'best_f1': 65.02821180555556
- 'best_f1_thresh': 0.0

#### Subset "movies" (subjqa)

- 'exact': 41.37931034482759
- 'f1': 44.96441783142732
- 'total': 261
---
- 'HasAns_exact': 2.0833333333333335
- 'HasAns_f1': 11.830344312526334
- 'HasAns_total': 96
---
- 'NoAns_exact': 64.24242424242425
- 'NoAns_f1': 64.24242424242425
- 'NoAns_total': 165
---
- 'best_exact': 63.2183908045977
- 'best_exact_thresh': 0.0
- 'best_f1': 63.2183908045977
- 'best_f1_thresh': 0.0

#### Subset "restaurants" (subjqa)

- 'exact': 35.2059925093633
- 'f1': 42.16978504051939
- 'total': 267
---
- 'HasAns_exact': 4.065040650406504
- 'HasAns_f1': 19.18156590096482
- 'HasAns_total': 123
---
- 'NoAns_exact': 61.80555555555556
- 'NoAns_f1': 61.80555555555556
- 'NoAns_total': 144
---
- 'best_exact': 53.93258426966292
- 'best_exact_thresh': 0.0
- 'best_f1': 54.74471845258363
- 'best_f1_thresh': 0.0

#### Subset "electronics" (subjqa)

- 'exact': 42.35294117647059
- 'f1': 47.397424488898295
- 'total': 255
---
- 'HasAns_exact': 4.081632653061225
- 'HasAns_f1': 17.207584129276142
- 'HasAns_total': 98
---
- 'NoAns_exact': 66.24203821656052
- 'NoAns_f1': 66.24203821656052
- 'NoAns_total': 157
---
- 'best_exact': 61.568627450980394
- 'best_exact_thresh': 0.0
- 'best_f1': 61.568627450980394
- 'best_f1_thresh': 0.0

#### Subset "grocery" (subjqa)

- 'exact': 37.15596330275229
- 'f1': 43.11579016957868
- 'total': 218
---
- 'HasAns_exact': 1.0204081632653061
- 'HasAns_f1': 14.277982213960732
- 'HasAns_total': 98
---
- 'NoAns_exact': 66.66666666666667
- 'NoAns_f1': 66.66666666666667
- 'NoAns_total': 120
---
- 'best_exact': 55.04587155963303
- 'best_exact_thresh': 0.0
- 'best_f1': 55.04587155963303
- 'best_f1_thresh': 0.0

#### Subset "Tripadvisor" (subjqa)

- 'exact': 20.0
- 'f1': 34.13426260363784
- 'total': 230
---
- 'HasAns_exact': 5.633802816901408
- 'HasAns_f1': 28.527326752371167
- 'HasAns_total': 142
---
- 'NoAns_exact': 43.18181818181818
- 'NoAns_f1': 43.18181818181818
- 'NoAns_total': 88
---
- 'best_exact': 38.26086956521739
- 'best_exact_thresh': 0.0
- 'best_f1': 38.26086956521739
- 'best_f1_thresh': 0.0